In [1]:
# model_name = "distilbert-base-uncased"  # or "bert-base-uncased"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModel.from_pretrained(model_name)

#%pip install streamlit
#%pip install faiss-cpu

import streamlit as st
import torch
from transformers import AutoModel, AutoTokenizer
from sentence_transformers import SentenceTransformer, util
import faiss

import numpy as np
import os
os.system("streamlit run mpq_chatbot.py &")

# Load DistilBERT-based embedding model
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Sample MPQ questions
mpq_questions = [
    "Describe your pain in your own words.",
    "Is the pain throbbing, shooting, stabbing, or burning?",
    "How intense is your pain on a scale of 1-10?",
    "Does the pain get worse with movement or at rest?",
    "How does the pain affect your emotions?",
]

# Predefined answers to common patient questions
faq = {
    "What is this questionnaire for?": "The McGill Pain Questionnaire helps doctors assess the nature of your pain.",
    "What does 'throbbing' mean?": "Throbbing pain is like a steady or pulsing ache.",
    "Can I skip a question?": "Yes, but answering all questions helps provide a full picture of your pain.",
}

# Encode FAQ for similarity search
faq_questions = list(faq.keys())
faq_embeddings = embedding_model.encode(faq_questions, convert_to_tensor=True)
index = faiss.IndexFlatL2(faq_embeddings.shape[1])
index.add(faq_embeddings.cpu().numpy())

# Initialize session state
if "step" not in st.session_state:
    st.session_state["step"] = 0
if "responses" not in st.session_state:
    st.session_state["responses"] = []

if "submitted" not in st.session_state:
    st.session_state["submitted"] = False

# Streamlit UI
st.title("McGill Pain Questionnaire Facilitator")

# Display current question
if st.session_state["step"] < len(mpq_questions):
    question = mpq_questions[st.session_state["step"]]
    st.subheader(question)

    user_input = st.text_input("Your response:", "")

    if st.button("Submit"):
        if user_input.strip():
            # Check if user asked a question
            input_embedding = embedding_model.encode(user_input, convert_to_tensor=True)
            D, I = index.search(input_embedding.cpu().numpy().reshape(1, -1), 1)
            if D[0][0] < 0.5:  # Threshold for similarity
                st.write(f"🤖 {faq[faq_questions[I[0][0]]]}")  # Answer detected question
            else:
                st.session_state["responses"].append((question, user_input))
                st.session_state["step"] += 1
                st.session_state["submitted"] = True  # Mark as submitted to prevent re-submission
                st.experimental_rerun() 

# Show summary when complete
if st.session_state["step"] >= len(mpq_questions):
    st.subheader("Summary of Your Responses")
    for q, r in st.session_state["responses"]:
        st.write(f"**{q}**\n{r}\n")
    st.success("Thank you for completing the questionnaire!")



/home/bhux/mikayla/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: mpq_chatbot.py


AttributeError: module 'faiss' has no attribute 'IndexFlatL2'

In [ ]:
import streamlit as st 
import torch
from transformers import AutoModel, AutoTokenizer
from sentence_transformers import SentenceTransformer, util
import faiss
import numpy as np

torch.classes.__path__ = []

# Load DistilBERT-based embedding model
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Sample MPQ questions
mpq_questions = [
    "Describe your pain in your own words.",
    "Is the pain throbbing, shooting, stabbing, or burning?",
    "How intense is your pain on a scale of 1-10?",
    "Does the pain get worse with movement or at rest?",
    "How does the pain affect your emotions?",
]

# Predefined answers to common patient questions
faq = {
    "What is this questionnaire for?": "The McGill Pain Questionnaire helps doctors assess the nature of your pain.",
    "What does 'throbbing' mean?": "Throbbing pain is like a steady or pulsing ache.",
    "Can I skip a question?": "Yes, but answering all questions helps provide a full picture of your pain.",
}

# Encode FAQ for similarity search
faq_questions = list(faq.keys())
faq_embeddings = embedding_model.encode(faq_questions, convert_to_tensor=True)
index = faiss.IndexFlatL2(faq_embeddings.shape[1])
index.add(faq_embeddings.cpu().numpy())

# Initialize session state
if "responses" not in st.session_state:
    st.session_state["responses"] = {}

st.title("McGill Pain Questionnaire")

# Use a form for input
with st.form(key="mpq_form"):
    responses = {}
    for question in mpq_questions:
        responses[question] = st.text_input(question, value="", key=question)

    submitted = st.form_submit_button("Submit")

if submitted:
    # Process responses
    for question, user_input in responses.items():
        if user_input.strip():
            input_embedding = embedding_model.encode(user_input, convert_to_tensor=True)
            D, I = index.search(input_embedding.cpu().numpy().reshape(1, -1), 1)
            if D[0][0] < 0.5:  # Threshold for similarity
                st.write(f"🤖 {faq[faq_questions[I[0][0]]]}")  # Answer detected question
            else:
                st.session_state["responses"][question] = user_input

    # Show summary
    st.subheader("Summary of Your Responses")
    for q, r in st.session_state["responses"].items():
        st.write(f"**{q}**\n{r}\n")
    
    st.success("Thank you for completing the questionnaire!")


In [ ]:
import streamlit as st 
import torch
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

torch.classes.__path__ = []

# Load DistilBERT-based embedding model
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Sample MPQ questions
mpq_questions = [
    "Describe your pain in your own words.",
    "Is the pain throbbing, shooting, stabbing, or burning?",
    "How intense is your pain on a scale of 1-10?",
    "Does the pain get worse with movement or at rest?",
    "How does the pain affect your emotions?",
]

# Predefined answers to common patient questions
faq = {
    "What is this questionnaire for?": "The McGill Pain Questionnaire helps doctors assess the nature of your pain.",
    "What does 'throbbing' mean?": "Throbbing pain is like a steady or pulsing ache.",
    "Can I skip a question?": "Yes, but answering all questions helps provide a full picture of your pain.",
}

# Encode FAQ for similarity search
faq_questions = list(faq.keys())
faq_embeddings = embedding_model.encode(faq_questions, convert_to_tensor=True)
index = faiss.IndexFlatL2(faq_embeddings.shape[1])
index.add(faq_embeddings.cpu().numpy())

# Initialize session state
if "step" not in st.session_state:
    st.session_state["step"] = 0
if "responses" not in st.session_state:
    st.session_state["responses"] = {}

st.title("McGill Pain Questionnaire")

# Check if questionnaire is completed
if st.session_state["step"] < len(mpq_questions):
    question = mpq_questions[st.session_state["step"]]
    st.subheader(question)

    # Chat interaction
    if "chat_history" not in st.session_state:
        st.session_state["chat_history"] = []

    for msg in st.session_state["chat_history"]:
        st.chat_message(msg["role"]).write(msg["content"])

    # User input
    user_input = st.chat_input("Type your response or ask a question...")

    if user_input:
        # Store chat history
        st.session_state["chat_history"].append({"role": "user", "content": user_input})

        # Check if user asked a question
        input_embedding = embedding_model.encode(user_input, convert_to_tensor=True)
        D, I = index.search(input_embedding.cpu().numpy().reshape(1, -1), 1)

        if D[0][0] < 0.5:  # User asked an FAQ
            answer = faq[faq_questions[I[0][0]]]
            st.session_state["chat_history"].append({"role": "bot", "content": answer})
            st.chat_message("bot").write(answer)
        else:
            # Save valid response and proceed to next question
            st.session_state["responses"][question] = user_input
            st.session_state["step"] += 1
            st.session_state["chat_history"] = []  # Clear chat for next question
            st.rerun()

else:
    # Show summary of responses
    st.subheader("Summary of Your Responses")
    for q, r in st.session_state["responses"].items():
        st.write(f"**{q}**\n{r}\n")
    
    st.success("Thank you for completing the questionnaire!")
